## User Input: Number of Clusters

In [ ]:
# number of clusters for each component (fixed)
# Note: There are 19 unique component/s for this dataset. 
num_clusters = 5

In [ ]:
# import packages
import pandas as pd
import pickle
from sklearn.decomposition import PCA

# Load grouped data by component from saved pickle

In [ ]:
with open("DummyPreproccessedForDoc2Vec.pickle", "rb") as pickle_file:
    grouped_data = pickle.load(pickle_file)

In [ ]:
grouped_data

# Examine by Subgroup

In [ ]:
# Select MCH subgroup to perform TF-IDF vectorization and k-means clustering
groups = grouped_data.keys()

# Vectorization: Doc2Vec

In [ ]:
# Doc to Vec
import gensim
import numpy as np
def tagged_document(list_of_lists):
    for i, list_of_words in enumerate(list_of_lists):
        yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])
        
# corpus = list(tagged_document(df['token']))

In [ ]:
group_X = {}
for g in groups:
    corpus = list(tagged_document(grouped_data[g]['Requirement']))
    d2v = gensim.models.doc2vec.Doc2Vec(vector_size=200, dm=0, min_count=2, epochs=50, seed=5)
    d2v.random.seed(5)
    d2v.build_vocab(corpus)
    d2v.random.seed(5)
    d2v.train(corpus, total_examples=d2v.corpus_count, epochs=d2v.epochs)
    # fit language model
    X = []
    for row in grouped_data[g]['Requirement']:
        d2v.random.seed(5)
        X.append(d2v.infer_vector(row.split(' ')))

    X = np.array(X)
    group_X[g] = X
    print('Word embeddings shape: ', end=' ')
    print(X.shape)

In [ ]:
group_X
with open("VectorizationDoc2Vec.pickle", "wb") as pickle_file:
    pickle.dump(group_X, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

# Principal Component Analysis

In [ ]:
# for component_name in list(groups):
#     # up to 95% of total variance
#     pca = PCA(n_components=0.95, svd_solver='full')
#     prin_comp = pca.fit_transform(group_X[component_name])
#     cumsum_variance = np.cumsum(pca.explained_variance_)
    
#     print(f'Component Name = {component_name}:')

#     print(f'# of eigenvalues (principal components) needed to reach '
#           f'{100*pca.n_components}% of explained variance: {pca.n_components_}')

#     print(f'Cumulative explained variance per principal component: '
#           f'{cumsum_variance[:4]} ... {cumsum_variance[-4:]} \n')

# K-means on all Subgroups

In [ ]:
'''
Cluster
'''
from sklearn.cluster import KMeans
import numpy as np

for g in groups:
    
    #FIXME: Each subgroup will have a different optimal K clusters... 
    # num_clusters = 6 # defined on top cell
    if len(grouped_data[g]) < num_clusters:
        num_clusters = 1
        
    
    model = KMeans(n_clusters=num_clusters, init='k-means++', random_state=5).fit(group_X[g])
    sizes = np.array(np.unique(model.labels_, return_counts=True))[1]
    print('Cluster sizes: ', end=' ')
    print(sizes)

    grouped_data[g]['predicted'] = model.labels_
    grouped_data[g].head()

In [ ]:
with open("ClustersDoc2Vec.pickle", "wb") as pickle_file:
    pickle.dump(grouped_data, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)